In [2]:
import sys
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import tensorflow as tf

In [1]:
from sherlock import helpers
from sherlock.features.preprocessing import extract_features, convert_string_lists_to_lists, prepare_feature_extraction
from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock
from  profiling import generate_header
from  profiling import identify_header
from  profiling import preprocess_before_count
from  profiling import filter_categories
from  profiling import get_var_category
from  profiling import process_df
from  profiling import generate_df
from  profiling import make_clickable
from  profiling import read_info

In [3]:
import numbers
import json
#'pip install git+git://github.com/clintval/gender_predictor.git'
from gender_predictor import GenderPredictor
import itertools

## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [4]:
helpers.download_data()
prepare_feature_extraction()

Data was downloaded.
Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


# Find all csv files

In [5]:
import os
from glob import glob
PATH = "/Users/max/Desktop/Cs576_DataProfiling/"
EXT = "*.csv"

In [6]:
all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
info = pd.DataFrame(columns = ['Dataset_ID','Age','Gender','Race','Ethnicity','Country','MUP','Uncovered Attributes','Report'])

In [7]:
for i in all_csv_files:
    dataset_ID = i.split('/')[-1]
    tmp_samples = pd.read_csv(i)
    # Generate PDF report
    profile = ProfileReport(tmp_samples, title=dataset_ID, explorative=True)
    output_path = "report/" + dataset_ID +".html"
    profile.to_file(output_path)
    header = identify_header(i)
    sensitive = ['age','area','nation','nationality','sex','gender','ethnicity','race']
    candidate = []
    # Generate header
    predicted_labels = generate_header(i)
    if not header:
        tmp_samples.columns = predicted_labels
    # Track whether these attributes exist
    name = gender = age = country = race = ethnicity = False
    for i in zip(tmp_samples.columns, predicted_labels):
        if i[0].lower() in sensitive or i[1].lower() in sensitive:
            candidate.append(i[0])
        if i[0].lower() in ['name','first_name','first name'] or i[1].lower() in ['name','first_name','first name']:
            name = i[0]
        if i[0].lower() in ['sex','gender'] or i[1].lower() in ['sex','gender']:
            gender = i[0]
        if i[0].lower() in ['age']:
            age = i[0]
        if i[0].lower() in ['country'] or i[1].lower() in ['country']:
            country = i[0]
        if i[0].lower() in ['race'] or i[1].lower() in ['race']:
            race = i[0]
        if i[0].lower() in ['ethnicity'] or i[1].lower() in ['ethnicity']:
            ethnicity = i[0]
    tmp, gender = preprocess_before_count(tmp_samples,name,gender,age)
    mup, uncover = process_df(tmp,candidate)
    tmp_info = generate_df(dataset_ID,age,gender,race,ethnicity,country,mup,uncover)
    #tmp_info = tmp_info.style.format({'Report': make_clickable})
    info = pd.concat([info, tmp_info], axis=0, sort=False)
info = info.style.format({'Report': make_clickable})
info.to_csv(r'info.csv', index = False)

W1204 10:26:37.521627 4627594752 deprecation.py:506] From /Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1204 10:26:37.523149 4627594752 deprecation.py:506] From /Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1204 10:26:37.527594 4627594752 deprecation.py:506] From /Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init_

100%|██████████| 15/15 [00:00<00:00, 398.46it/s]


Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


/Users/max/Desktop/Cs576_DataProfiling/notebooks/profiling.py:156: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ele in j:


# Read the output info.xlsx

In [12]:
read_info('info.xlsx')

AttributeError: 'Styler' object has no attribute 'to_csv'